# All-Convolutional Net for the CIFAR-10 dataset

In this notebook, the model in this notebook is inspired by the All Convolutional Net model

- J. T. Springenberg et al., 2015, Striving for Simplicity: The All Convolutional Net

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint

import time

/home/arthur/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Preparation

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('X shape :', x_train.shape)
print(len(x_train), 'train samples')
print(len(x_test), 'test samples')

X shape : (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
x_train_mean = np.mean(x_train, axis=0)
x_train_std = np.std(x_train, axis=0)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_train_mean)/x_train_std

n_y = 10
y_train = keras.utils.to_categorical(y_train, n_y)
y_test = keras.utils.to_categorical(y_test, n_y)

# Model

In [6]:
input_shape = x_train.shape[1:]
activation = 'relu'
padding = 'same'
l2_reg = keras.regularizers.l2(0.0001)

model = Sequential()

model.add(Conv2D(64, (3,3), padding=padding, input_shape=input_shape, activation=activation, 
                 kernel_regularizer=l2_reg))
model.add(Conv2D(64, (3,3), padding=padding, strides=2, activation=activation, kernel_regularizer=l2_reg))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding=padding, activation=activation, kernel_regularizer=l2_reg))
model.add(Conv2D(128, (3,3), padding=padding, strides=2, activation=activation, kernel_regularizer=l2_reg))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), padding=padding, activation=activation, kernel_regularizer=l2_reg))
model.add(Conv2D(256, (3,3), padding=padding, strides=2, activation=activation,kernel_regularizer=l2_reg))
model.add(AveragePooling2D(4,4))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(n_y, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 256)         295168    
__________

# Model training

In [7]:
optimizer = keras.optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)
model.compile(optimizer, keras.losses.categorical_crossentropy, ['accuracy'])

shift = 4/32
generator = ImageDataGenerator(width_shift_range=shift, height_shift_range=shift, 
                               horizontal_flip=True)

batch_size = 64
n_steps = x_train.shape[0]//batch_size

save_path = './Model_trained/All_CNN_cifar10.h5'
ckeckpoint = ModelCheckpoint(save_path, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=np.sqrt(0.1), patience=5, verbose=1, min_delta=0.01)


t0 = time.time()
model.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=n_steps, 
                    epochs=200, validation_data=(x_test, y_test), 
                    callbacks=[ckeckpoint, reduce_lr])
print('Total training time : %.3f s' %(time.time()-t0))

Epoch 1/200
781/781 [==============================] - 18s 23ms/step - loss: 2.0883 - acc: 0.2224 - val_loss: 1.7986 - val_acc: 0.3438

Epoch 00001: val_acc improved from -inf to 0.34380, saving model to ./Model_trained/All_CNN_cifar10.h5
Epoch 2/200
781/781 [==============================] - 18s 23ms/step - loss: 1.7766 - acc: 0.3655 - val_loss: 1.5692 - val_acc: 0.4494

Epoch 00002: val_acc improved from 0.34380 to 0.44940, saving model to ./Model_trained/All_CNN_cifar10.h5
Epoch 3/200
781/781 [==============================] - 17s 22ms/step - loss: 1.6628 - acc: 0.4275 - val_loss: 1.5296 - val_acc: 0.4854

Epoch 00003: val_acc improved from 0.44940 to 0.48540, saving model to ./Model_trained/All_CNN_cifar10.h5
Epoch 4/200
781/781 [==============================] - 17s 22ms/step - loss: 1.6194 - acc: 0.4639 - val_loss: 1.4472 - val_acc: 0.5294

Epoch 00004: val_acc improved from 0.48540 to 0.52940, saving model to ./Model_trained/All_CNN_cifar10.h5
Epoch 5/200
781/781 [==============

781/781 [==============================] - 18s 23ms/step - loss: 0.8264 - acc: 0.8058 - val_loss: 0.8432 - val_acc: 0.8146

Epoch 00038: val_acc did not improve from 0.83300
Epoch 39/200
781/781 [==============================] - 17s 22ms/step - loss: 0.8245 - acc: 0.8080 - val_loss: 0.8051 - val_acc: 0.8236

Epoch 00039: val_acc did not improve from 0.83300
Epoch 40/200
781/781 [==============================] - 17s 22ms/step - loss: 0.8236 - acc: 0.8090 - val_loss: 0.8138 - val_acc: 0.8196

Epoch 00040: val_acc did not improve from 0.83300
Epoch 41/200
781/781 [==============================] - 17s 22ms/step - loss: 0.8172 - acc: 0.8131 - val_loss: 0.7583 - val_acc: 0.8369

Epoch 00041: val_acc improved from 0.83300 to 0.83690, saving model to ./Model_trained/All_CNN_cifar10.h5
Epoch 42/200
781/781 [==============================] - 18s 23ms/step - loss: 0.8182 - acc: 0.8107 - val_loss: 0.8029 - val_acc: 0.8219

Epoch 00042: val_acc did not improve from 0.83690
Epoch 43/200
781/781 [

781/781 [==============================] - 18s 23ms/step - loss: 0.4881 - acc: 0.8979 - val_loss: 0.5352 - val_acc: 0.8904

Epoch 00078: val_acc improved from 0.88520 to 0.89040, saving model to ./Model_trained/All_CNN_cifar10.h5
Epoch 79/200
781/781 [==============================] - 18s 24ms/step - loss: 0.4714 - acc: 0.9044 - val_loss: 0.5199 - val_acc: 0.8955

Epoch 00079: val_acc improved from 0.89040 to 0.89550, saving model to ./Model_trained/All_CNN_cifar10.h5
Epoch 80/200
781/781 [==============================] - 18s 23ms/step - loss: 0.4674 - acc: 0.9044 - val_loss: 0.5289 - val_acc: 0.8944

Epoch 00080: val_acc did not improve from 0.89550
Epoch 81/200
781/781 [==============================] - 17s 22ms/step - loss: 0.4596 - acc: 0.9070 - val_loss: 0.5369 - val_acc: 0.8933

Epoch 00081: val_acc did not improve from 0.89550
Epoch 82/200
781/781 [==============================] - 17s 22ms/step - loss: 0.4599 - acc: 0.9067 - val_loss: 0.5533 - val_acc: 0.8865

Epoch 00082: val

781/781 [==============================] - 18s 22ms/step - loss: 0.4006 - acc: 0.9233 - val_loss: 0.4990 - val_acc: 0.9015

Epoch 00118: val_acc did not improve from 0.90290
Epoch 119/200
781/781 [==============================] - 18s 23ms/step - loss: 0.4046 - acc: 0.9213 - val_loss: 0.4991 - val_acc: 0.9013

Epoch 00119: val_acc did not improve from 0.90290
Epoch 120/200
781/781 [==============================] - 18s 22ms/step - loss: 0.4040 - acc: 0.9217 - val_loss: 0.4984 - val_acc: 0.9012

Epoch 00120: val_acc did not improve from 0.90290

Epoch 00120: ReduceLROnPlateau reducing learning rate to 3.1622778678900043e-06.
Epoch 121/200
781/781 [==============================] - 18s 23ms/step - loss: 0.4035 - acc: 0.9223 - val_loss: 0.4982 - val_acc: 0.9013

Epoch 00121: val_acc did not improve from 0.90290
Epoch 122/200
781/781 [==============================] - 18s 22ms/step - loss: 0.4037 - acc: 0.9223 - val_loss: 0.4983 - val_acc: 0.9015

Epoch 00122: val_acc did not improve from 

KeyboardInterrupt: 

# Final Evaluation

In [8]:
best_model = keras.models.load_model(save_path)
print('Best model test accuracy :', best_model.evaluate(x_test, y_test, batch_size=64)[1])

10000/10000 [==============================] - 1s 140us/step
Best model test accuracy : 0.9029


# Conclusion

$90 \%$ accuracy correspond to the results of the paper. It's a good result when we take into account the simplicity of the model, constitued of only convolution layer.